In [36]:
import requests
import json
import pandas as pd
import numpy as np

In [55]:
## 1번 노드 상태 받기!! >> ㅇㅋㅇㅋ

headers = {'Content-Type' : 'application/json; charset=utf-8'}
a = requests.get("http://localhost:5000/chain", headers=headers)
status_json = json.loads(a.text)
status_json['chain']

[{'index': 1,
  'previous_hash': 1,
  'proof': 100,
  'timestamp': 1652922951.8196206,
  'transactions': []},
 {'index': 2,
  'previous_hash': 'b900abd84614cb9fbb28b2dc76d71361d799fb91cf9f0b578b7e34e79a12e1ff',
  'proof': 88384,
  'timestamp': 1652922954.1798725,
  'transactions': [{'amount': 3,
    'recipient': 'zzz',
    'sender': 'abc',
    'timestamp': 1652922953.1153772},
   {'amount': 1003,
    'recipient': 'yyy',
    'sender': 'def',
    'timestamp': 1652922953.121301},
   {'amount': 143,
    'recipient': 'ccc',
    'sender': 'tyu',
    'timestamp': 1652922953.1262085},
   {'amount': 0.1,
    'recipient': 'node_5000',
    'sender': 'master',
    'timestamp': 1652922954.1748877}]},
 {'index': 3,
  'previous_hash': 'e57e43e30dfb39a73a09e953f5a192a7555d71415abd2a07c69b92cd6e3e3c80',
  'proof': 100,
  'timestamp': 1652922960.5413048,
  'transactions': [{'amount': 0.1,
    'recipient': 'node_5000',
    'sender': 'master',
    'timestamp': 1652922954.1772385},
   {'amount': 3,
    're

In [38]:
tx_amount_l = []
tx_sender_l = []
tx_reciv_l  = []
tx_time_l   = []

for chain_index in range(len(status_json['chain'])):
    chain_tx = status_json['chain'][chain_index]['transactions']
    for each_tx in range(len(chain_tx)):
        tx_amount_l.append(chain_tx[each_tx]['amount'])
        tx_sender_l.append(chain_tx[each_tx]['sender'])
        tx_reciv_l.append(chain_tx[each_tx]['recipient'])
        tx_time_l.append(chain_tx[each_tx]['timestamp'])
        
df_tx = pd.DataFrame()
df_tx['timestamp'] = tx_time_l  
df_tx['sender'] = tx_sender_l 
df_tx['recipient'] = tx_reciv_l
df_tx['amount'] = tx_amount_l   
df_tx

,timestamp,sender,recipient,amount
0,1.652923e+09,abc,zzz,3.0
1,1.652923e+09,def,yyy,1003.0
2,1.652923e+09,tyu,ccc,143.0
3,1.652923e+09,master,node_5000,0.1
4,1.652923e+09,master,node_5000,0.1
5,1.652923e+09,abc2,zzz2,3.0
6,1.652923e+09,zzz2,def2,3.0
7,1.652923e+09,def2,yyy2,1003.0
8,1.652923e+09,tyu2,ccc2,143.0
9,1.652923e+09,master,node_5001,0.1


In [48]:
df_sended = pd.DataFrame(df_tx.groupby('sender')['amount'].sum()).reset_index()
df_sended.columns = ['user','sended_amount']
df_received= pd.DataFrame(df_tx.groupby('recipient')['amount'].sum()).reset_index()
df_received.columns = ['user','received_amount']
df_received

,user,received_amount
0,ccc,143.0
1,ccc2,143.0
2,def2,3.0
3,node_5000,0.2
4,node_5001,0.1
5,yyy,1003.0
6,yyy2,1003.0
7,zzz,3.0
8,zzz2,3.0


In [54]:
df_balance = pd.DataFrame()
df_balance['user'] = np.unique(np.append(df_tx['sender'].unique(), df_tx['recipient'].unique()))
df_balance = pd.merge(df_balance, df_sended, on ='user', how='left')
df_balance = pd.merge(df_balance, df_received, on ='user', how='left')
df_balance =df_balance.fillna(0)
df_balance['final_balance']=  df_balance['sended_amount'] - df_balance['received_amount']
df_balance

,user,sended_amount,received_amount,final_balance
0,abc,3.0,0.0,3.0
1,abc2,3.0,0.0,3.0
2,ccc,0.0,143.0,-143.0
3,ccc2,0.0,143.0,-143.0
4,def,1003.0,0.0,1003.0
5,def2,1003.0,3.0,1000.0
6,master,0.3,0.0,0.3
7,node_5000,0.0,0.2,-0.2
8,node_5001,0.0,0.1,-0.1
9,tyu,143.0,0.0,143.0
